In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy

In [2]:
csv_file = "Resources/countryLockdowndates.csv"
lockdown_df = pd.read_csv(csv_file)
lockdown_df.head()

,Country/Region,Province,Date,Type,Reference
0,Afghanistan,NaN,24/03/2020,Full,https://www.thestatesman.com/world/afghan-govt...
1,Albania,NaN,08/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
2,Algeria,NaN,24/03/2020,Full,https://www.garda.com/crisis24/news-alerts/325...
3,Andorra,NaN,16/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...
4,Angola,NaN,24/03/2020,Full,https://en.wikipedia.org/wiki/2020_coronavirus...


In [3]:
csv_file_1 = "Resources/WHR20_DataForFigure2.1.csv"
happy_df = pd.read_csv(csv_file_1)
happy_df

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.8087,0.031156,7.869766,7.747634,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,Denmark,Western Europe,7.6456,0.033492,7.711245,7.579955,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,Switzerland,Western Europe,7.5599,0.035014,7.628528,7.491272,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,Iceland,Western Europe,7.5045,0.059616,7.621347,7.387653,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,Norway,Western Europe,7.4880,0.034837,7.556281,7.419719,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Central African Republic,Sub-Saharan Africa,3.4759,0.115183,3.701658,3.250141,6.625160,0.319460,45.200001,0.640881,0.082410,0.891807,1.972317,0.041072,0.000000,0.000000,0.292814,0.253513,0.028265,2.860198
149,Rwanda,Sub-Saharan Africa,3.3123,0.052425,3.415053,3.209547,7.600104,0.540835,61.098846,0.900589,0.055484,0.183541,1.972317,0.343243,0.522876,0.572383,0.604088,0.235705,0.485542,0.548445
150,Zimbabwe,Sub-Saharan Africa,3.2992,0.058674,3.414202,3.184198,7.865712,0.763093,55.617260,0.711458,-0.072064,0.810237,1.972317,0.425564,1.047835,0.375038,0.377405,0.151349,0.080929,0.841031
151,South Sudan,Sub-Saharan Africa,2.8166,0.107610,3.027516,2.605684,7.425360,0.553707,51.000000,0.451314,0.016519,0.763417,1.972317,0.289083,0.553279,0.208809,0.065609,0.209935,0.111157,1.378751


In [4]:
cleaned_lockdown = lockdown_df[['Country/Region', 'Province', 'Date', 'Type']].copy()
cleaned_lockdown.head()

,Country/Region,Province,Date,Type
0,Afghanistan,NaN,24/03/2020,Full
1,Albania,NaN,08/03/2020,Full
2,Algeria,NaN,24/03/2020,Full
3,Andorra,NaN,16/03/2020,Full
4,Angola,NaN,24/03/2020,Full


In [5]:
rds_connection_string = "postgres:postgres@localhost:5433/ETL_SQL"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [6]:
engine.table_names()

['lockdown', 'happiness']

In [7]:
renamed_lockdown = cleaned_lockdown.rename(columns={'Country/Region':'country', 'Province':'province', 'Date':'date', 'Type':'type'})
renamed_lockdown

,country,province,date,type
0,Afghanistan,NaN,24/03/2020,Full
1,Albania,NaN,08/03/2020,Full
2,Algeria,NaN,24/03/2020,Full
3,Andorra,NaN,16/03/2020,Full
4,Angola,NaN,24/03/2020,Full
...,...,...,...,...
302,Venezuela,NaN,16/03/2020,Full
303,Vietnam,NaN,19/03/2020,Full
304,West Bank and Gaza,NaN,05/03/2020,Full
305,Zambia,NaN,NaN,None


In [8]:
renamed_lockdown["province"].fillna("", inplace = True) 
renamed_lockdown

,country,province,date,type
0,Afghanistan,,24/03/2020,Full
1,Albania,,08/03/2020,Full
2,Algeria,,24/03/2020,Full
3,Andorra,,16/03/2020,Full
4,Angola,,24/03/2020,Full
...,...,...,...,...
302,Venezuela,,16/03/2020,Full
303,Vietnam,,19/03/2020,Full
304,West Bank and Gaza,,05/03/2020,Full
305,Zambia,,NaN,None


In [9]:
renamed_lockdown.dtypes

country     object
province    object
date        object
type        object
dtype: object

In [10]:
renamed_lockdown['date'] = pd.to_datetime(renamed_lockdown["date"], format="%d/%m/%Y")


In [11]:
renamed_lockdown['date'] = renamed_lockdown['date'].dt.strftime('%Y/%m/%d')

In [12]:
cleaned_happy = happy_df[['Country name', 'Regional indicator', 'Logged GDP per capita', "Social support", "Healthy life expectancy", 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']].copy()
cleaned_happy.head()

,Country name,Regional indicator,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,Finland,Western Europe,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445
1,Denmark,Western Europe,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489
2,Switzerland,Western Europe,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728
3,Iceland,Western Europe,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710
4,Norway,Western Europe,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218


In [13]:
renamed_happy = cleaned_happy.rename(columns={'Country name': 'country_name', 'Regional indicator': 'regional_indicator', 'Logged GDP per capita': 'logged_gdp_per_capita', "Social support": 'social_support', "Healthy life expectancy": 'healthy_life_expectancy', 'Freedom to make life choices':'freedom_to_make_life_choices', 'Generosity': 'generosity', 'Perceptions of corruption': 'perceptions_of_corruption'})
renamed_happy

,country_name,regional_indicator,logged_gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption
0,Finland,Western Europe,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445
1,Denmark,Western Europe,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489
2,Switzerland,Western Europe,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728
3,Iceland,Western Europe,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710
4,Norway,Western Europe,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218
...,...,...,...,...,...,...,...,...
148,Central African Republic,Sub-Saharan Africa,6.625160,0.319460,45.200001,0.640881,0.082410,0.891807
149,Rwanda,Sub-Saharan Africa,7.600104,0.540835,61.098846,0.900589,0.055484,0.183541
150,Zimbabwe,Sub-Saharan Africa,7.865712,0.763093,55.617260,0.711458,-0.072064,0.810237
151,South Sudan,Sub-Saharan Africa,7.425360,0.553707,51.000000,0.451314,0.016519,0.763417


In [15]:
renamed_happy.to_sql(name='happiness', con=engine, if_exists='append', index=False)

In [16]:
renamed_lockdown.to_sql(name='lockdown', con=engine, if_exists='append', index=False)


In [19]:
pd.read_sql_query('select * from lockdown', con=engine).head()

,country,province,date,type
0,Afghanistan,,2020-03-24,Full
1,Albania,,2020-03-08,Full
2,Algeria,,2020-03-24,Full
3,Andorra,,2020-03-16,Full
4,Angola,,2020-03-24,Full


In [18]:
pd.read_sql_query('select * from happiness', con=engine).head()

,country_name,regional_indicator,logged_gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption
0,Pakistan,South Asia,8.482727,0.689062,58.253136,0.734834,0.044900,0.745705
1,Finland,Western Europe,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445
2,Denmark,Western Europe,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489
3,Switzerland,Western Europe,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728
4,Iceland,Western Europe,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710
